In [23]:
import psycopg2
import psycopg2.extras

import json
#!pip install json-lines
import json_lines
from pathlib import Path
import pandas as pd    
#!pip install memory_profiler  
import time                 # timing operations
import memory_profiler      # managing memory usage
from memory_profiler import memory_usage


# Посмотреть JSON

In [10]:
with open("sample.json","r", encoding="utf-8") as file:
    data = json.loads(file.read())
    for item in data:
        print(item)

{'url': 'https://apkbr.ru/node/9529', 'title': 'Лъэмыжыр къызэIуахыжащ', 'publication_date': '2023-12-19T13:05:57+00:00', 'content': '«Кавказ» зи фIэщыгъэ Р-217 къэралпсо автомашинэ ', 'author': 'ШУРДЫМ Динэ.'}
{'url': 'https://apkbr.ru/node/9528', 'title': 'Къэбэрдей жэщтеуэм иуэрэдымрэ ', 'publication_date': '2023-12-19T13:05:57+00:00', 'content': '«Кавказ» зи фIэщыгъэ Р-217 къэралпсо автомашинэ ', 'author': 'БАХЪСЭН Азэмэт.'}
{'url': 'https://apkbr.ru/node/9527', 'title': 'ЛIыхъужьым и партэ', 'publication_date': '2023-12-19T11:52:55+00:00', 'content': '«Фэеплъ\n\xa0\nНалшык къалэм и курыт еджапIэ №1-м щагъэув', 'author': None}


In [11]:
json_obj = pd.read_json('sample.json')
json_obj 

,url,title,publication_date,content,author
0,https://apkbr.ru/node/9529,Лъэмыжыр къызэIуахыжащ,2023-12-19T13:05:57+00:00,«Кавказ» зи фIэщыгъэ Р-217 къэралпсо автомашинэ,ШУРДЫМ Динэ.
1,https://apkbr.ru/node/9528,Къэбэрдей жэщтеуэм иуэрэдымрэ,2023-12-19T13:05:57+00:00,«Кавказ» зи фIэщыгъэ Р-217 къэралпсо автомашинэ,БАХЪСЭН Азэмэт.
2,https://apkbr.ru/node/9527,ЛIыхъужьым и партэ,2023-12-19T11:52:55+00:00,«Фэеплъ\n \nНалшык къалэм и курыт еджапIэ №1-м...,None


# Посмотреть JSONL

In [36]:
json_lines = pd.read_json('apkbr_ru.jsonl', lines=True )
json_lines.head()

,url,title,publication_date,content,author
0,https://apkbr.ru/node/9529,Лъэмыжыр къызэIуахыжащ,2023-12-19T13:05:57+00:00,«Кавказ» зи фIэщыгъэ Р-217 къэралпсо автомашин...,ШУРДЫМ Динэ.
1,https://apkbr.ru/node/9528,Къэбэрдей жэщтеуэм иуэрэдымрэ хъыбарымрэ,2023-12-19T12:03:06+00:00,"Жэщтеуэмэ, уэр, и хъыбар пэжыр, аредэ, вжесIэн...",Тедзэным хуэзыгъэхьэзырар ТАБЫЩ Муратщ.
2,https://apkbr.ru/node/9527,ЛIыхъужьым и партэ,2023-12-19T11:52:55+00:00,Фэеплъ\n \nНалшык къалэм и курыт еджапIэ №1-м ...,БАХЪСЭН Азэмэт.
3,https://apkbr.ru/node/9506,«ХъуэпсапIэхэм я псей»,2023-12-17T08:00:44+00:00,Къэбэрдей-Балъкъэрым дыгъуасэ щрагъэжьащ «Хъуэ...,None
4,https://apkbr.ru/node/9491,Хы псэущхьэхэр мащIэ хъуащ,2023-12-15T14:30:41+00:00,Иужьрей зэманым псэущхьэ зэмылIэужьыгъуэхэр щI...,ФЫРЭ Анфисэ.


In [37]:
print(type(json_obj ), type(json_lines))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


# Положить всё в одну таблицу

In [14]:
def create_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS texts;
        CREATE UNLOGGED TABLE texts (
            url                 TEXT,
            title               TEXT,
            publication_date	TEXT,
            content             TEXT,
            author              TEXT
        );""")

# look at the documentation of PSQL (UNLOGGED TABLE vs TEMP)

In [32]:
def df_to_table(df, table, cursor):

    if len(df) > 0:
        df_columns = list(df) # ['url', 'title', 'publication_date', 'content', 'author']
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = f"INSERT INTO {table} ({columns}) {values}"
        cursor.execute("truncate " + table + ";")  #avoiding uploading duplicate data
        cursor = connection.cursor()
        psycopg2.extras.execute_batch(cursor, insert_stmt, df.values)
    connection.commit()

In [38]:
try:
    connection = psycopg2.connect(
        host="127.0.0.1",
        user="postgres",
        password="postgres123",
        database="education"
    )
    connection.autocommit = True

    with connection.cursor() as cursor:
        cursor.execute("SELECT version();")
        print(f"Server version: {cursor.fetchone()}")

    with connection.cursor() as cursor:
        create_table(cursor)
    
    with connection.cursor() as cursor:
        df_to_table(json_lines,'texts', cursor)

except Exception as _ex:
    print(f"[INFO] Error while working with PostgreSQL", _ex)

finally:
    if connection:
        connection.close()
        #print("[INFO] PostgreSQL connection closed")

Server version: ('PostgreSQL 10.23, compiled by Visual C++ build 1800, 32-bit',)


# Положить в разные таблицы

1. Таблица "Источники" (поля: id_источника, источник)
2. Таблица "Предложения" (поля: id_текста, текст, id_источника)
3. Таблица "Слова" (поля: id_слова, слово)
4. Таблица-связь "Слова" и "Предложения" (поля: id_слова, id_предложения)
7. Таблица-связь "Слова" и "Источники". (поля: id_слова, id_источника, кол-во слова в источнике) 